<center>
    <h1>   
        ==#== TEXT CLASSIFICATION ON COVID-19 TWEETS ==#==
    </h1>
</center>
 
<center><img src="https://www.marketplace.org/wp-content/uploads/2020/03/GettyImages-1205872351-e1584559616344.jpg?w=600" alt="COVID-19" width="600" ></center>

<center> <h3> Please Upvote if you like the work 😊 </h3></center>

<center>Don't hesitate to give your comment on this notebook. It gives me motivation to improve the analysis in the future</center>

# Introduction
This program is used to classify tweets into 5 classifications: Extremely Negative, Negative, Neutral, Positive, and Extremely Positive. This code is useful for understanding the public's perception of a particular event. Some codes are inspired by [Himanshutripathi's Notebook](https://www.kaggle.com/himanshutripathi/covid-19-tweets-analysis-97-accuracy).

# Contents
Contents:

* [1. Load Packages](#1)
* [2. Load Dataset](#2)
* [3. Drop NaN and duplicates values](#3)
* [4. Do Explanatory Data Analysis](#4)
* [5. Mapping Sentiment](#5) 
* [6. Modelling](#6)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

<a id="1"></a>
    
<font size="+2" color="indigo"><b>1. Load Packages</b></font><br>

In [ ]:
# importing all necessery modules for Word Cloud
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
import pandas as pd

# Modelling
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize

import re,string,unicodedata
from keras.preprocessing import text, sequence
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.model_selection import train_test_split
from string import punctuation
import keras
from keras.models import Sequential
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import tensorflow as tf

<a id="2"></a>
    
<font size="+2" color="indigo"><b>2. Load Dataset</b></font><br>



In [ ]:
test_df = pd.read_csv("/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv")
train_df = pd.read_csv("/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv")

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
display(train_df.isnull().sum())
print("====///====")
display(train_df.duplicated().sum())
print("====///====")
display(train_df.info())

> Many of null values of train_df are located in "location" column

In [ ]:
display(test_df.isnull().sum())
print("====///====")
display(test_df.duplicated().sum())
print("====///====")
display(test_df.info())

> Also, the null value are located in "location" column for this dataset

<a id="3"></a>
    
<font size="+2" color="indigo"><b>3. Drop NaN and duplicates values</b></font><br>


In [ ]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

train_df.drop_duplicates(inplace= True)
test_df.drop_duplicates(inplace=True)

<a id="4"></a>
    
<font size="+2" color="indigo"><b>4. Do Explanatory Data Analysis</b></font><br>



In [ ]:
import seaborn as sns

In [ ]:
train_df.head()

In [ ]:
sns.set(rc={'figure.figsize':(11,5)})
sns.countplot(train_df['Sentiment'])

In [ ]:
# Python program to generate WordCloud
  ## Based on https://www.geeksforgeeks.org/generating-word-cloud-python/

comment_words = ''
stopwords = set(STOPWORDS)
  
# iterate through the csv file
for val in train_df.OriginalTweet:
      
    # typecaste each val to string
    val = str(val)
  
    # split the value
    tokens = val.split()
      
    # Converts each token into lowercase
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
      
    comment_words += " ".join(tokens)+" "
  
wordcloud = WordCloud(width = 800, height = 800,
                background_color ='white',
                stopwords = stopwords,
                min_font_size = 10).generate(comment_words)
  
# plot the WordCloud image                       
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)
  
plt.show()

In [ ]:
# Ref: https://stackoverflow.com/questions/64719706/cleaning-twitter-data-pandas-python
# Ref: https://www.kaggle.com/ragnisah/text-data-cleaning-tweets-analysis
# Ref: https://www.kaggle.com/himanshutripathi/covid-19-tweets-analysis-97-accuracy

import nltk
import re,string,unicodedata
# load stop words
stopword = nltk.corpus.stopwords.words('english')
def clean(text):

    #     remove urls
    text = re.sub(r'http\S+', " ", text)

    #     remove mentions
    text = re.sub(r'@\w+',' ',text)

    #     remove hastags
    text = re.sub(r'#\w+', ' ', text)

    #     remove digits
    text = re.sub(r'\d+', ' ', text)

    #     remove html tags
    text = re.sub('r<.*?>',' ', text)
    
    #     remove stop words 
    text = text.split()
    text = " ".join([word for word in text if not word in stopword])
    
      
    return text

train_df['OriginalTweet'] = train_df['OriginalTweet'].apply(lambda x: clean(x))
test_df['OriginalTweet'] = test_df['OriginalTweet'].apply(lambda x: clean(x))

In [ ]:
# Delete the data having length of Original Tweet <1
train_df=train_df[~train_df['OriginalTweet'].str.len().lt(1)]
test_df=test_df[~test_df['OriginalTweet'].str.len().lt(1)]

<a id="5"></a>
    
<font size="+2" color="indigo"><b>5. Mapping Sentiment</b></font><br>

In [ ]:
train_df['Sentiment'].unique()

In [ ]:
#sentiment_mapping['Sentiment']=train_df['Sentiment'].factorize()[0]
sentiment_changer = {"Extremely Negative":0, "Negative":1,"Neutral":2, "Positive":3,"Extremely Positive":4}
train_df['Sentiment'] = train_df['Sentiment'].map(sentiment_changer)
test_df['Sentiment'] = test_df['Sentiment'].map(sentiment_changer)

In [ ]:
train_df

<a id="6"></a>
    
<font size="+2" color="indigo"><b>6. Modelling</b></font><br>


In [ ]:
train_df.head()

In [ ]:
X = train_df['OriginalTweet'].copy()
y = train_df['Sentiment'].copy()

#X_test = train_df['OriginalTweet'].copy()
#y_test = train_df['Sentiment'].copy()

In [ ]:

max_len = np.max(X.apply(lambda x :len(x)))
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15,random_state = 0)

vocab_length = len(tokenizer.word_index) + 1

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

X_train = pad_sequences(X_train, maxlen=max_len, padding='post')
X_test = pad_sequences(X_test, maxlen=max_len, padding='post')

In [ ]:
embedding_dim=16
print("Vocab length:", vocab_length)
print("Max sequence length:", max_len)

In [ ]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_length, embedding_dim, input_length=max_len),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256, return_sequences=True)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(5, activation='softmax')
])
# opt = tf.keras.optimizers.Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy',optimizer="adam",metrics=['accuracy'])

In [ ]:

tf.keras.utils.plot_model(model)

In [ ]:
num_epochs = 10
from keras.utils import to_categorical

y_train = to_categorical(y_train, 5)
y_test = to_categorical(y_test, 5)
history = model.fit(X_train, y_train, epochs=num_epochs,validation_data=(X_test, y_test))

In [ ]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
epochs = range(len(acc))

plt.plot(epochs, acc,'b',label='training acc')
plt.plot(epochs, val_acc, 'r', label='validation acc')
plt.legend()
plt.show()


plt.plot(epochs, loss,'b',label='training loss')
plt.plot(epochs, val_loss, 'r', label='validation loss')
plt.legend()
plt.show()

<a id="7"></a>
    
<font size="+2" color="indigo"><b>7. Conclusion</b></font><br>

> It can be seen that the level of accuracy of the model to the training data increases with increasing epoch. However, it can also be seen that the accuracy rate of the validation data is in the range of 75%.

# Please Upvote if you like the work. Thank you 😊